# Analyze Distribution

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

SITE = "googlemap"

df = pd.read_csv(f"../../database/reviews_{SITE}.csv")

# 텍스트 길이 계산
df["review_len"] = df["text"].astype(str).str.len()

# 날짜 변환
df["date"] = pd.to_datetime(df["date"], format="%Y.%m.%d", errors="coerce")

# 저장 폴더
os.makedirs(f"plots/{SITE}", exist_ok=True)

## Rating dist.

In [3]:
plt.figure(figsize=(6, 4))
sns.countplot(x="rating", data=df)
plt.title(f"{SITE} - Rating Distribution")
plt.xlabel("Rating")
plt.ylabel("Count")
plt.savefig(f"plots/{SITE}/{SITE}_rating_dist.png", bbox_inches="tight")
plt.close()

## Review Length dist.

In [4]:
plt.figure(figsize=(6, 4))
sns.histplot(df["review_len"], bins=30, kde=True)
plt.title(f"{SITE} - Review Length Distribution")
plt.xlabel("Review Length")
plt.ylabel("Frequency")
plt.savefig(f"plots/{SITE}/catchtable_reviewlen_dist.png", bbox_inches="tight")
plt.close()


## Date dist.

In [5]:
plt.figure(figsize=(10, 4))
df["date"].dropna().dt.to_period("M").value_counts().sort_index().plot()
plt.title(f"{SITE} - Monthly Review Count")
plt.xlabel("Month")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.savefig(f"plots/{SITE}/catchtable_date_dist.png", bbox_inches="tight")
plt.close()


# Outlier Detection

In [ ]:
today = pd.Timestamp.today()

# 1. 형식 이상치 (숫자로 변환되지 않는 값)
rating_str = df["rating"].astype(str)
rating_numeric = pd.to_numeric(rating_str, errors="coerce")
invalid_format_rating = df[rating_numeric.isna()]

# 2. 범위 이상치 (1 ~ 5를 벗어난 값)
valid_numeric_rating = df[rating_numeric.notna()].copy()
valid_numeric_rating["rating"] = rating_numeric.dropna()
out_of_range_rating = valid_numeric_rating[
    (valid_numeric_rating["rating"] < 1) | (valid_numeric_rating["rating"] > 5)
]
too_short = df[df["review_len"] < 3]
too_long = df[df["review_len"] > 1800]  
too_old = df[df["date"] < "2015-01-01"]
future_date = df[df["date"] > today]
non_parsable_date = df[df["date"].isna()]

print("──────────── Outlier Report ────────────")
print(f"[별점 형식 오류]          {len(invalid_format_rating)}개")
print(f"[별점 범위 이상치]        {len(out_of_range_rating)}개")  
print(f"[너무 짧은 리뷰]          {len(too_short)}개")
print(f"[너무 긴 리뷰]            {len(too_long)}개")
print(f"[너무 오래된 날짜]        {len(too_old)}개")
print(f"[미래 날짜]              {len(future_date)}개")
print(f"[날짜 형식 오류] (NaT)    {len(non_parsable_date)}개")
print("────────────────────────────────────────")


──────────── Outlier Report ────────────
[별점 형식 오류]          0개
[별점 범위 이상치]        0개
[너무 짧은 리뷰]          0개
[너무 긴 리뷰]            0개
[너무 오래된 날짜]        0개
[미래 날짜]              0개
[날짜 형식 오류] (NaT)    38개
────────────────────────────────────────
